In [179]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import random
import warnings
warnings.filterwarnings("ignore")

In [239]:
columns = []
data = []
test_req = requests.get("https://www.pro-football-reference.com/boxscores/201709100cin.htm")
soup = BeautifulSoup(test_req.content, "html.parser")
scoring_table = soup.find_all('h2')[0]
if 'Scoring' not in str(scoring_table):
    print('Error')
test_table = scoring_table.find_next('table')
if columns == []:
    columns = [th.text.strip() for th in test_table.find_all('th')][:6]
elif columns[2:] != [th.text.strip() for th in test_table.find_all('th')]:
    print(f'issue in season {season} week {week}' )
for row in test_table.find('tbody').find_all('tr'):
    row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)

In [242]:
def clean_df(df):
    df = df.drop(columns=['Tm','Detail'])

    df['num_seconds'] = df['Time'].str.split(":").str[0].astype(int)*60+df['Time'].str.split(":").str[1].astype(int)
    
    index_list = []
    if '1' in df['Quarter'].unique():
        index_1 = df.loc[df['Quarter']=='1',:].index[0]
        index_list.append(index_1)
    if '2' in df['Quarter'].unique():
        index_2 = df.loc[df['Quarter']=='2',:].index[0]
        index_list.append(index_2)
    if '3' in df['Quarter'].unique():
        index_3 = df.loc[df['Quarter']=='3',:].index[0]
        index_list.append(index_3)
    if '4' in df['Quarter'].unique():
        index_4 = df.loc[df['Quarter']=='4',:].index[0]
        index_list.append(index_4)
        
    if '1' in df['Quarter'].unique():
        lowest_num = min([x for x in index_list if x > index_1])
        df.loc[index_1:lowest_num-1,'num_seconds'] = df.loc[index_1:lowest_num-1,'num_seconds'] + 15*60*3
    if '2' in df['Quarter'].unique():
        lowest_num = min([x for x in index_list if x > index_2])
        df.loc[index_2:lowest_num-1,'num_seconds'] = df.loc[index_2:lowest_num-1,'num_seconds'] + 15*60*2
    if '3' in df['Quarter'].unique():
        if '4' in df['Quarter'].unique():
            lowest_num = min([x for x in index_list if x > index_3])
        else:
            lowest_num = len(df)
        df.loc[index_3:lowest_num-1,'num_seconds'] = df.loc[index_3:lowest_num-1,'num_seconds'] + 15*60*1

    df.iloc[:,2] = df.iloc[:,2].astype(int)
    df.iloc[:,3] = df.iloc[:,3].astype(int)

    df['within_3'] = (abs(df.iloc[:,3] - df.iloc[:,2])<=3).astype(int)
    return df

df = clean_df(pd.DataFrame(data, columns=columns))
df

,Quarter,Time,BAL,CIN,num_seconds,within_3
0,1,4:44,3,0,2984,1
1,2,1:52,10,0,1912,0
2,,1:28,17,0,1888,0
3,3,1:06,20,0,966,0


In [205]:
def calculate_prop(df):
    secs_remaining = 3600
    num = 0
    counting = True
    for i in range(len(df)):
        if counting == True:
            if df.iloc[i,:]['within_3'] == 1:
                continue
            else:
                num = num + (secs_remaining - df.iloc[i,:]['num_seconds'])
                counting = False
                secs_remaining = df.iloc[i,:]['num_seconds']
        else:
            if df.iloc[i,:]['within_3'] == 0:
                continue
            else:
                counting = True
                secs_remaining = df.iloc[i,:]['num_seconds']
    if counting == True:
        num = num + secs_remaining
    prop = num/3600
    return prop
calculate_prop(df)

0.6291666666666667

In [206]:
def calculate_switches(df):
    df['differential'] = df.iloc[:,2] - df.iloc[:,3]
    return np.sum(np.abs(np.diff(df['differential']>0))) + sum(df['differential'] == 0)
calculate_switches(df)

9

In [167]:
full_df = pd.read_csv("../data/closeness_links.csv")
full_df

,Season,Week,Day,Date,Time (ET),Unnamed: 5,Favorite,Score,Spread,Unnamed: 9,Underdog,Over/Under,Notes,GameHub/RedZone,Game thread link?,Post-game thread link?,Game thread,Post game thread,Subscribers (in thousands)
0,2017,1,Thu,7-Sep-17,8:30,@,New England Patriots,L 27-42,L -8,NaN,Kansas City Chiefs,O 47.5,NaN,NaN,NaN,NaN,https://www.reddit.com/r/nfl/comments/6yr619/g...,https://www.reddit.com/r/nfl/comments/6ysc10/p...,640
1,2017,1,Sun,10-Sep-17,1:00,@,Buffalo Bills,W 21-12,W -7,NaN,New York Jets,U 42,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97il/g...,https://www.reddit.com/r/nfl/comments/6zajs1/p...,640
2,2017,1,Sun,10-Sep-17,1:00,NaN,Atlanta Falcons,W 23-17,L -6.5,@,Chicago Bears,U 48,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97gn/g...,https://www.reddit.com/r/nfl/comments/6zajuc/p...,640
3,2017,1,Sun,10-Sep-17,1:00,@,Cincinnati Bengals,L 0-20,L -2.5,NaN,Baltimore Ravens,U 41.5,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97hl/g...,https://www.reddit.com/r/nfl/comments/6zajr0/p...,640
4,2017,1,Sun,10-Sep-17,1:00,NaN,Pittsburgh Steelers,W 21-18,L -10,@,Cleveland Browns,U 47,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97l9/g...,https://www.reddit.com/r/nfl/comments/6zajwi/p...,640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2022,18,Sun,8-Jan-23,4:25,@,Philadelphia Eagles,W 22-16,L -16,NaN,New York Giants,U 42,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u055/...,https://www.reddit.com/r/nfl/comments/106zb92/...,3197
1563,2022,18,Sun,8-Jan-23,4:25,@,San Francisco 49ers,W 38-13,W -14,NaN,Arizona Cardinals,O 39.5,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u07e/...,https://www.reddit.com/r/nfl/comments/106z77a/...,3197
1564,2022,18,Sun,8-Jan-23,4:25,@,Seattle Seahawks,W 19-16 (OT),L -6,NaN,Los Angeles Rams,U 41.5,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u06w/...,https://www.reddit.com/r/nfl/comments/106zix9/...,3197
1565,2022,18,Sun,8-Jan-23,4:25,NaN,Dallas Cowboys,L 6-26,L -7,@,Washington Commanders,U 40,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u05s/...,https://www.reddit.com/r/nfl/comments/106zgna/...,3197


In [168]:
full_df['Date'] = pd.to_datetime(full_df['Date'])
nfl_teams_code = {
    'arizona cardinals': 'crd',
    'atlanta falcons': 'atl',
    'baltimore ravens': 'bal',
    'buffalo bills': 'buf',
    'carolina panthers': 'car',
    'chicago bears': 'chi',
    'cincinnati bengals': 'cin',
    'cleveland browns': 'cle',
    'dallas cowboys': 'dal',
    'denver broncos': 'oti',
    'detroit lions': 'det',
    'green bay packers': 'gnb',
    'houston texans': 'htx',
    'indianapolis colts': 'clt',
    'jacksonville jaguars': 'jax',
    'kansas city chiefs': 'kan',
    'las vegas raiders': 'rai',
    'los angeles chargers': 'sdg',
    'los angeles rams': 'ram',
    'miami dolphins': 'mia',
    'minnesota vikings': 'min',
    'new england patriots': 'nwe',
    'new orleans saints': 'nor',
    'new york giants': 'nyg',
    'new york jets': 'nyj',
    'philadelphia eagles': 'phi',
    'pittsburgh steelers': 'pit',
    'san francisco 49ers': 'sfo',
    'seattle seahawks': 'sea',
    'tampa bay buccaneers': 'tam',
    'tennessee titans': 'ten',
    'washington football team': 'was',
    'washington commanders': 'was'
}
full_df['link_text_fav'] = "https://www.pro-football-reference.com/boxscores/" + \
                    full_df['Date'].dt.strftime('%Y%m%d').astype(str) + '0' + \
                    full_df['Favorite'].str.lower().replace(nfl_teams_code) + ".htm"
full_df['link_text_dog'] = "https://www.pro-football-reference.com/boxscores/" + \
                    full_df['Date'].dt.strftime('%Y%m%d').astype(str) + '0' + \
                    full_df['Underdog'].str.lower().replace(nfl_teams_code) + ".htm"

In [149]:
# test_req = requests.get(full_df['link_text_fav'][1400])
# soup = BeautifulSoup(test_req.content, "html.parser")
# scoring_table = soup.find_all('h2')[0]
# scoring_table

<h2>Scoring</h2>

In [221]:
problematic_rows = []
props = []
switches = []
for index, row in full_df[:5].iterrows():
    if index % 100 == 0:
        print(f"Scraped {index}")
    test_req = requests.get(row['link_text_fav'])
    soup = BeautifulSoup(test_req.content, "html.parser")
    scoring_table = soup.find_all('h2')[0]
    if 'Scoring' not in str(scoring_table):
        test_req = requests.get(row['link_text_dog'])
        soup = BeautifulSoup(test_req.content, "html.parser")
        scoring_table = soup.find_all('h2')[0]
        if 'Scoring' not in str(scoring_table):
            problematic_rows.append(index)
            props.append(np.NaN)
            switches.append(np.NaN)
            time.sleep(3)
            continue
    try:
        test_table = scoring_table.find_next('table')
        columns = [th.text.strip() for th in test_table.find_all('th')][:6]
        data = []
        for row in test_table.find('tbody').find_all('tr'):
            row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
            data.append(row_data)
        temp_df = pd.DataFrame(data, columns=columns)
        
        temp_df = clean_df(temp_df)
        
        props.append(calculate_prop(temp_df))
        
        switches.append(calculate_switches(temp_df))
        
    except:
        problematic_rows.append(index)
        props.append(np.NaN)        
    time.sleep(3)

Scraped 0


In [222]:
problematic_rows

[1, 3]

In [224]:
switches

[4, 6, 3]

In [238]:
full_df.iloc[3,:]['link_text_fav']

'https://www.pro-football-reference.com/boxscores/201709100cin.htm'

In [227]:
row = full_df.iloc[1,:]#['link_text_fav'], full_df.iloc[1415,:]['link_text_dog']
test_req = requests.get(row['link_text_fav'])
soup = BeautifulSoup(test_req.content, "html.parser")
scoring_table = soup.find_all('h2')[0]
if 'Scoring' not in str(scoring_table):
    print('x')
test_table = scoring_table.find_next('table')
columns = [th.text.strip() for th in test_table.find_all('th')][:6]
data = []
for row in test_table.find('tbody').find_all('tr'):
    row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)
temp_df = pd.DataFrame(data, columns=columns)
temp_df = clean_df(temp_df)
temp_df
calculate_prop(temp_df)
calculate_switches(temp_df)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [230]:
temp_df['Quarter'].unique()#.loc[temp_df['Quarter']=='1',:]

array(['2', '', '3', '4'], dtype=object)